In [1]:
fp = r"D:\workspace\hw-memory\example\spec_example\spec2006_470.lbm\lbm_part.asm"

In [2]:
from sample.cfg import proc_identify
from sample.isa import Instruction, AddrMode
from typing import Tuple
from sample.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 00000000000005e8
.plt 0000000000000600
__cxa_finalize@plt 0000000000000620
__libc_start_main@plt 0000000000000630
__gmon_start__@plt 0000000000000640
abort@plt 0000000000000650
printf@plt 0000000000000660
_start 0000000000000670
call_weak_fn 00000000000006a8
deregister_tm_clones 00000000000006c0
register_tm_clones 00000000000006f0
__do_global_dtors_aux 0000000000000730
frame_dummy 0000000000000778
main 000000000000077c
initializerandommesh 00000000000007dc
swapMesh 000000000000086c
SET_FLAG 00000000000008ac
__libc_csu_init 0000000000000920
__libc_csu_fini 00000000000009a0
_fini 00000000000009a4


In [3]:
from graphviz import Digraph
from sample.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output\\procedures.svg'

In [4]:
from sample.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output\\call_graph.svg'

In [5]:
from sample.cfg import TCfg

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

tcfg.build_loop_hrchy()
for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))

for n in tcfg.all_nodes:
    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
    print("{} {}".format(n.name, temp))


l2 (in l1 children []): n7->n8 ['n15', 'n6', 'n8', 'n7']
l1 (in l0 children ['l2']): n9->n10 ['n15', 'n5', 'n7', 'n9', 'n6', 'n8', 'n10']
l0 (in None children ['l1']): n11->n12 ['n15', 'n5', 'n4', 'n7', 'n9', 'n11', 'n6', 'n8', 'n10', 'n12']
n0 None
n1 None
n2 None
n3 None
n4 l0
n5 l1
n6 l2
n7 l2
n8 l2
n9 l1
n10 l1
n11 l0
n12 l0
n13 None
n14 None
n15 l2
n16 None
